## Notes
- Comment faire la traduction ? Comment faire pour les NER tags

In [1]:
import torch
print(torch.cuda.is_available())

print(torch.cuda.device_count())
print(torch.cuda.current_device())
torch.cuda.device(1)
torch.cuda.get_device_name(0)

True
2
0


'NVIDIA TITAN Xp'

In [2]:
model_name = {}
model_name['fr'] = 'camembert-base'
model_name['en'] = 'roberta-base'
model_translation = {}
model_translation['fr_en'] = 'Helsinki-NLP/opus-mt-fr-en'
model_translation['en_fr'] = 'Helsinki-NLP/opus-mt-en-fr'
dataset_name = 'wikiann'

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset_en = load_dataset(dataset_name, 'en')
dataset_fr = load_dataset(dataset_name, 'fr')

tokenizer = {}
tokenizer['fr'] = AutoTokenizer.from_pretrained(model_name['fr'])
tokenizer['en'] = AutoTokenizer.from_pretrained(model_name['en'], add_prefix_space=True)

Found cached dataset wikiann (/home/desponds/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset wikiann (/home/desponds/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
label_names = dataset_en["train"].features["ner_tags"].feature.names

In [5]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(examples, tokenizer):
    tokenized_samples = tokenizer.batch_encode_plus(examples["tokens"], is_split_into_words=True, return_overflowing_tokens=True)

    sample_map = tokenized_samples.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        tokenized_samples[key] = [values[i] for i in sample_map]
    #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
    #so the new keys [input_ids, labels (after adjustment)]
    #can be added to the datasets dict for each train test validation split
    total_adjusted_labels = []
    print(len(tokenized_samples["input_ids"]))
    for k in range(0, len(tokenized_samples["input_ids"])):
        prev_wid = -1
        word_ids_list = tokenized_samples.word_ids(batch_index=k)
        existing_label_ids = examples["ner_tags"][k]
        i = -1
        adjusted_label_ids = []
     
    for wid in word_ids_list:
        if(wid is None):
            adjusted_label_ids.append(-100)
        elif(wid!=prev_wid):
            i = i + 1
            adjusted_label_ids.append(existing_label_ids[i])
            prev_wid = wid
        else:
            label_name = label_names[existing_label_ids[i]]
            adjusted_label_ids.append(existing_label_ids[i])
            
        total_adjusted_labels.append(adjusted_label_ids)
    tokenized_samples["labels"] = total_adjusted_labels
    return tokenized_samples

In [6]:
def tokenize_and_align_labels(examples, language, label_all_tokens = True):
    tokenized_inputs = tokenizer[language](examples["tokens"], truncation=True, is_split_into_words=True)
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
tokenized_fr = {}
for split in dataset_fr :
    tokenized_fr[split] = dataset_fr[split].map(lambda examples : tokenize_and_align_labels(examples, 'fr'),batched=True)
    tokenized_fr[split] = tokenized_fr[split].remove_columns(dataset_fr["train"].column_names)


Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-15e4d95708d5dc9f.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-2a9093d41c7dec89.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-29fcdc61481c0254.arrow


In [8]:
tokenized_en = {}
for split in dataset_en :
    tokenized_en[split] = dataset_en[split].map(lambda examples : tokenize_and_align_labels(examples, 'en'),batched=True)

Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-b707a0950af3d31d.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-9d8ea39e43a49cbd.arrow
Loading cached processed dataset at /home/desponds/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e/cache-914886c5b5f307bf.arrow


In [9]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
        if(k not in flattened_results.keys()):
            flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

/tmp/ipykernel_126099/4005921667.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [10]:
from transformers import DataCollatorForTokenClassification
data_collator = {}
data_collator['fr'] = DataCollatorForTokenClassification(tokenizer['fr'])
data_collator['en'] = DataCollatorForTokenClassification(tokenizer['en'])

model = {}
model['fr'] = AutoModelForTokenClassification.from_pretrained(model_name['fr'])
model['en'] = AutoModelForTokenClassification.from_pretrained(model_name['en'])

training_args = {}
training_args['fr'] = TrainingArguments(
    output_dir="/data/desponds/data/NER/trainer_fr/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy='epoch',
    remove_unused_columns=False
)


trainer = {}
trainer['fr'] = Trainer(
    model=model['fr'],
    args=training_args['fr'],
    train_dataset=tokenized_fr["train"].select(range(100)),
    eval_dataset=tokenized_fr["validation"],
    data_collator=data_collator['fr'],
    tokenizer=tokenizer['fr'],
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas

In [11]:
tokenized_fr['train'][0],dataset_fr['train'][0]

({'input_ids': [5, 503, 5511, 427, 491, 2601, 1051, 3424, 6],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, 5, 5, 5, 5, 6, 6, 6, -100]},
 {'tokens': ['Saxifrage', 'faux', 'Orpin'],
  'ner_tags': [5, 6, 6],
  'langs': ['fr', 'fr', 'fr'],
  'spans': ['LOC: Saxifrage faux Orpin']})

In [14]:
import os
import wandb
os.environ["WANDB_API_KEY"]="fb22019a857cb5bd0b79ccd93abf9de65ea6a2ee"
os.environ["WANDB_ENTITY"]="Mathiponds"
os.environ["WANDB_PROJECT"]="question_answering"
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [15]:
trainer['fr'].train()

***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12
  Number of trainable parameters = 110032898
You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/desponds/anaconda3/lib/python3.9/site-packages/transformers/models/camembert/modeling_camembert.py", line 1280, in forward
    outputs = self.roberta(
  File "/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/desponds/anaconda3/lib/python3.9/site-packages/transformers/models/camembert/modeling_camembert.py", line 903, in forward
    encoder_outputs = self.encoder(
  File "/data/desponds/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py"

## Translation

In [19]:
dataset_fr['train'][145]

{'tokens': ["'", "''", 'Honda', "''", "'"],
 'ner_tags': [0, 0, 3, 0, 0],
 'langs': ['fr', 'fr', 'fr', 'fr', 'fr'],
 'spans': ['ORG: Honda']}

In [ ]:
from helper import translate_fr_en
def translate_fr_en_qa(example):
    example['tokens'] = translate_fr_en(' '.join(example['tokens'])).split(' ')
    
    return example

In [ ]:
import pickle
# Translate the test split of the french dataset
translated_fr_en = dataset_fr['valid'].map(translate_fr_en_qa)

with open('/data/desponds/data/NER/translated_dataset.pickle', 'wb') as handle:
    pickle.dump(translated_fr_en, handle)

## Evaluate

In [ ]:
trainer['fr'].predict(tokenized_fr['test'])

## Results

In [ ]:
import pandas as pd
data = {
    'task' : ['NER', 'NER', 'NER'],
    'dataset' : ['', '', ''],
    'translated' : ['no', 'no', 'yes'],
    'model'   : ['CamemBERT', 'Roberta', 'Roberta'],
    'test_loss' : [0.9777808785438538, 0.9427361488342285, 1.096827745437622],
    'test_accuracy' : [0.5972, 0.6044, 0.552]
}
results = pd.DataFrame(data)
results